In [1]:
CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, diameter_mm, series_uid, center_xys',
)

NameError: name 'namedtuple' is not defined

In [ ]:
@functools.lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool = True):
    mhd_list = glob.glob('D:/dataset/LUNA16DATASET/subset*/*.mhd')
    presentOnDisk_set = {os.path.split(p)[-1][:-4] for p in mhd_list}


In [ ]:
    diameter_dict = {}
        with open('D:/dataset/LUNA16DATASET/annotations.csv', 'r') as f:
            for row in list(csv.reader(f))[1:]:
                series_uid = row[0]
                annotationCenter_xyz = tuple([float(x) for x in row[1:4]])
                annotationDiameter_mm = float(row[4])

                diameter_dict.setdefault(series_uid, []).append((annotationCenter_xyz, annotationDiameter_mm))

    # 两个文件存储的中心点坐标距离是否超过了结节直径的四分之一
    candidateInfo_list = []
        with open('D:/dataset/LUNA16DATASET/candidates.csv', 'r') as f:
            for row in list(csv.reader(f))[1:]:
                series_uid = row[0]

                if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                    continue

                isNodule_bool = bool(int(row[4]))
                condidateCenter_xyz = tuple([float(x) for x in row[1:4]])

                candidateDiameter_mm = 0.0
                for annotation_tup in diameter_dict.get(series_uid, []):
                    annotationCenter_xyz, annotationDiameter_mm = annotation_tup
                    for i in range(3):
                        delta_mm = abs(condidateCenter_xyz[i] - annotationCenter_xyz[i])
                        if delta_mm > annotationDiameter_mm / 4:
                            break
                        else:
                            candidateDiameter_mm = annotationDiameter_mm
                            break
                candidateInfo_list.append(CandidateInfoTuple(
                    isNodule_bool,
                    candidateDiameter_mm,
                    series_uid,
                    condidateCenter_xyz,
                ))

In [ ]:
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

In [ ]:
# SimpleITK读取mhd文件
import SimpleITK as sitk

class Ct:
    def __init__(self, series_uid):
        mhd_path = glob.glob('D:/dataset/LUNA16DATASET/subset*/{}.mhd'.format(series_uid))[0]
        ct_mhd = sitk.ReadImage(mhd_path)
        ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

        ct_a.clip(-1000, 1000, ct_a)
        self.series_uid = series_uid
        self.hu_a = ct_a # hounsfield unit

        self.origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
        self.vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
        self.direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

In [ ]:
# 毫米为单位的坐标成为（X, Y, Z）坐标，以体素为单位的坐标成为（I, R, C）坐标
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])

def irc2xyz(coord_irc, origin_xyz, vxSize_xyz, direction_a):
    cri_a = np.array(coord_irc)[::-1]
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coords_xyz = (direction_a @ (cri_a * vxSize_a)) + origin_a
    return XyzTuple(*coords_xyz)

def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coords_xyz = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))


In [ ]:
def getRawCandidate(self, center_xyz, width_irc):
    center_irc = xyz2irc(center_xyz, self.origin_xyz, self.vxSize_xyz, self.direction_a)
    slice_list = []
    for axis, center_val in enumerate(center_irc):
        start_ndx = int(round(center_val - width_irc[axis] / 2))
        end_ndx = int(start_ndx + width_irc[axis])

        assert center_val >= 0 and center_val < self.hu_a.shape[axis], repr([self.series_uid, center_xyz, self.center_irc, self.origin_xyz, self.vxSize_xyz, center_irc, axis])

        if start_ndx < 0:
            start_ndx = 0
            end_ndx = int(width_irc[axis])
        if end_ndx > self.hu_a.shape[axis]:
            end_ndx = self.hu_a.shape[axis]
            start_ndx = int(self.hu_a.shape[axis] - width_irc[axis])
        slice_list.append(slice(start_ndx, end_ndx))

    ct_chunk = self.hu_a[tuple(slice_list)]

    return ct_chunk, center_irc

In [ ]:
# 编写Dataset方法
@functools.lru_cache(1, typed=True)
def getCt(series_uid):
    return Ct(series_uid)

@raw_cache.memoize(typed=True)
def getCtRawCandidate(series_uid, center_xyz, width_irc):
    ct = getCt(series_uid)
    ct_chunk, center_irc = ct.getRawCandidate(center_xyz, width_irc)
    return ct_chunk, center_irc

In [ ]:
def __len__(self):
    return len(self.candidateInfo_list)

def __getitem__(self, ndx):
    candidateInfo_tup = self.candidateInfo_list[ndx]

    width_irc = (32, 48, 48)

    candidate_a, center_irc = getCtRawCandidate(candidateInfo_tup.series_uid, candidateInfo.center_xyz, width_irc)
    candidate_t = torch.from_numpy(candidate_a)
    candidate_t = candidate_t.to(torch.float32)
    candidate_t = candidate_t.unsqueeze(0)

    pos_t = torch.tensor([not candidateInfo_tup.isNodule_bool, candidateInfo_tup.isNodule_bool], dtype=torch.long)

    return (candidate_t, pos_t, candidateInfo_tup.series_uid, torch.tensor(center_irc))

In [ ]:
# 分割训练集和验证集
class LunaDataset():
    def __init__(self, val_stride=0, isValSet_bool=None, series_uid=None):
        self.candidateInfo_list = copy.copy(getCandidateInfoList())
        if series_uid:
            self.candidateInfo_list = [x for x in self.candidateInfo_list if x.series_uid == series_uid]

        if isValSet_bool:
            assert val_stride > 0, val_stride
            self.candidateInfo_list = self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list
        elif val_stride > 0:
            del self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list

        log.info("{!r}: {} {} samples".format(self, len(self.candidateInfo_list), "validation" if isValSet_bool else "training"))